In [27]:
# import necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd


# define website
url='https://www.realestate.com.au/sold/in-pyrmont,+nsw+2009/list-1?activeSort=solddate&sourcePage=rea:homepage&sourceElement=suburb-select:recent%20searches%20tiles'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'}
response=requests.get(url=url,headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

In [10]:
# Implement a function to handle requests with retries
def make_request(url, max_retries=3):
    for _ in range(max_retries):
        response = requests.get(url=url,headers=headers)
        if response.status_code == 200:
            return response
        elif response.status_code == 429:
            # If rate-limited, wait and then retry
            time.sleep(5)  # Adjust the delay as needed
        else:
            print(f"Failed with status code: {response.status_code}")
            return None
response = make_request(url)
soup = BeautifulSoup(response.content, "html.parser")

AttributeError: 'NoneType' object has no attribute 'content'

In [3]:
# Check status of response website
def scrape_property_data(url):
    response=requests.get(url)
    if response.status_code !=200:
        return None
    soup = BeautifulSoup(response.content, "html.parser")
    address = soup.find('span')
    property_data = []
    property_list = soup.find_all("div", class_="residential-card__content")
    for property_item in property_list:
        address = property_item.find("span", class_="residential-card__address-heading").text.strip()
        sale_date = property_item.find("div", class_="PropertyCardLayout__StyledPipedContent-sc-1qkhjdh-0 dgCHTg").text.strip()
        amount = property_item.find("div", class_="residential-card__price").text.strip()
        if "Contact Agent" in amount:
            amount = None
        num_bedrooms = property_item.find("div", class_="View__PropertyDetail-sc-11ysrk6-0 eSRWKr").text.strip()
        num_bathrooms = property_item.find("div", class_="View__PropertyDetail-sc-11ysrk6-0 eSRWKr").text.strip()
        property_type = "Apartment"
        
        property_data.append({
            "Address": address,
            "Sale Date": sale_date,
            "Amount": amount,
            "Number of Bedrooms": num_bedrooms,
            "Number of Bathrooms": num_bathrooms,
            "Type": property_type
        })
    
    return property_data

# Scrape data from the base URL
property_data = scrape_property_data(url)

# Create a DataFrame from the scraped data
df = pd.DataFrame(property_data)

# Display the DataFrame
print(df)

Empty DataFrame
Columns: []
Index: []


In [14]:
property_list = soup.find_all("div", {'class':"residential-card__content-wrapper",'role':"presentation"})
print(property_list)

[]


In [15]:
property_list = soup.find_all("a", class_="details-link residential-card__details-link")
print(property_list)

[]


In [16]:
print(soup.text)

In [28]:
if response.status_code == 200:
    print(response.text)  # Print the content of the response
else:
    print(f'Request failed with status code: {response.status_code}')

Request failed with status code: 429
